In [ ]:
import numpy as np
import pennylane as qml
from pennylane import qchem


In [ ]:
pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00


In [ ]:
symbols = ["N", "H", "H", "H"]
coordinates = np.array([0,0,0,0,-0.9366,-0.3816,0.8121,0.4689,-0.3816,-0.8121,0.4689,-0.3816]) #referencia [2]

#charge i multiplicity definits abans i son iguals

electrons = 10
orbitals = 7
multiplicity = 1
charge = 0
basis_set = "sto-3g"
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)


print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of NH3")
print(H)

List of core orbitals: [0, 1, 2]
List of active orbitals: [3, 4, 5, 6]
Number of qubits: 8
Number of qubits required to perform quantum simulations: 8
Hamiltonian of NH3
-49.35524844390622 * I(0) + 0.3944125133384788 * Z(0) + 0.04136702824854272 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.04136702824854272 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.25129571416006485 * Z(2) + 0.1503515099054795 * (Z(0) @ Z(2)) + 0.019316634879539895 * (Y(2) @ Z(3) @ Y(4)) + 0.019316634879539895 * (X(2) @ Z(3) @ X(4)) + -0.0010810439267729525 * (Z(0) @ Y(2) @ Z(3) @ Y(4)) + -0.0010810439267729525 * (Z(0) @ X(2) @ Z(3) @ X(4)) + -0.48246857946267663 * Z(4) + 0.1230608387722581 * (Z(0) @ Z(4)) + -0.5289993691159496 * Z(6) + 0.13660539083989054 * (Z(0) @ Z(6)) + 0.39441251333847865 * Z(1) + 0.17892968352102617 * (Z(0) @ Z(1)) + 0.015931499155788755 * (Y(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.015931499155788755 * (X(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.010980654872647932 * (Y

In [ ]:
H

(
    -49.35524844390622 * I(0)
  + 0.3944125133384788 * Z(0)
  + 0.04136702824854272 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + 0.04136702824854272 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + 0.25129571416006485 * Z(2)
  + 0.1503515099054795 * (Z(0) @ Z(2))
  + 0.019316634879539895 * (Y(2) @ Z(3) @ Y(4))
  + 0.019316634879539895 * (X(2) @ Z(3) @ X(4))
  + -0.0010810439267729525 * (Z(0) @ Y(2) @ Z(3) @ Y(4))
  + -0.0010810439267729525 * (Z(0) @ X(2) @ Z(3) @ X(4))
  + -0.48246857946267663 * Z(4)
  + 0.1230608387722581 * (Z(0) @ Z(4))
  + -0.5289993691159496 * Z(6)
  + 0.13660539083989054 * (Z(0) @ Z(6))
  + 0.39441251333847865 * Z(1)
  + 0.17892968352102617 * (Z(0) @ Z(1))
  + 0.015931499155788755 * (Y(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + 0.015931499155788755 * (X(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + 0.010980654872647932 * (Y(0) @ X(1) @ X(2) @ Y(3))
  + -0.010980654872647932 * (Y(0) @ Y(1) @ X(2) @ X(3))
  + -0.010980654872647932 * (X(0) @ X(1) @ Y(2

In [ ]:
dev = qml.device("default.qubit", wires=qubits)
from pennylane import numpy as np
theta = np.array(0.0, requires_grad = True)
electrons = 2
hf = qml.qchem.hf_state(electrons,qubits)
def circuit(param,wires):
    qml.BasisState(hf, wires = wires)
    qml.DoubleExcitation(param,wires = [0,1,2,3])

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.2)

In [ ]:
@qml.qnode(dev, interface = "autograd")
def cost_fn(param):
    circuit(param,wires = range(qubits))
    return qml.expval(H)
# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 20
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    #if conv <= conv_tol:
       # break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")
energy_amoni = energy[-1]
print(energy_amoni)

Step = 0,  Energy = -51.17744080 Ha
Step = 2,  Energy = -51.17805603 Ha
Step = 4,  Energy = -51.17852924 Ha
Step = 6,  Energy = -51.17889305 Ha
Step = 8,  Energy = -51.17917267 Ha
Step = 10,  Energy = -51.17938753 Ha
Step = 12,  Energy = -51.17955259 Ha
Step = 14,  Energy = -51.17967938 Ha
Step = 16,  Energy = -51.17977676 Ha
Step = 18,  Energy = -51.17985154 Ha

Final value of the ground-state energy = -51.17988215 Ha

Optimal value of the circuit parameter = 0.1010
-51.17988214672228


In [ ]:
pip install qiskit-Aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qiskit_aer import Aer
from qiskit import transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.circuit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
#from qiskit.algorithms import NumPyEigensolver
import qiskit.primitives

In [ ]:
import numpy as np
from qiskit_aer import Aer
from qiskit import transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.quantum_info import SparsePauliOp
import qiskit.primitives

In [ ]:
import numpy as np
from qiskit_aer import Aer
from qiskit import transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.quantum_info import SparsePauliOp
import qiskit.primitives

# Define el Hamiltoniano como una lista de términos Pauli
hamiltonian_terms = [
    ("IIIIIIII", -49.35524844390622),
    ("ZZZZZZZZ", 0.3944125133384788),
    ("YZZZZZZY", 0.04136702824854272),
    ("XZZZZZZX", 0.04136702824854272),
    ("ZZIZIZZZ", 0.25129571416006485),
    ("ZZIZIZIZ", 0.1503515099054795),
    ("YYIZIYIZ", 0.019316634879539895),
    ("XXIZIXIZ", 0.019316634879539895),
    ("ZZYZZZYZ", -0.0010810439267729525),
    ("ZZXZZXZX", -0.0010810439267729525),
    ("ZZZZZZZZ", -0.48246857946267663),
    ("ZZZZZZIZ", 0.1230608387722581),
    ("ZZZZZZZZ", -0.5289993691159496),
    ("ZZZZZZZZ", 0.13660539083989054),
    ("IZZZZZZZ", 0.39441251333847865),
    ("ZZZZZZZI", 0.17892968352102617),
    ("YZZZZZYI", 0.015931499155788755),
    ("XZZZZZXI", 0.015931499155788755),
    ("YXZZZZXY", 0.010980654872647932),
    ("YYZZZZXX", -0.010980654872647932),
    ("XXZZZZYY", -0.010980654872647932),
    ("XYZZZZYY", 0.010980654872647932),
    ("YXXYYYXI", 0.000695477059732626),
    ("YYYYYYYY", 0.000695477059732626),
    ("XXXXXXXX", 0.000695477059732626),
    ("XYYYYXYY", 0.000695477059732626),
    ("YXYYYXYY", -0.000695477059732626),
    ("YYXXXZYY", 0.000695477059732626),
    ("XXYYXZYY", 0.000695477059732626),
    ("XYYXXZZZ", -0.000695477059732626),
    ("YYXXXXZZ", 0.000695477059732626),
    ("YXXZZYYZ", 0.000695477059732626),
    ("XYYXZYYZ", 0.000695477059732626),
    ("XZZZXYZZ", 0.007751954751065937),
    ("YZZZZYZZ", -0.007751954751065937),
    ("XZZZZXZZ", -0.007751954751065937),
    ("XYYXXZYY", 0.007751954751065937),
    ("YXXZZYYZ", 0.041367028248542735),
    ("ZZXZZZXZ", 0.015931499155788755),
    ("YZZZZYZZ", 0.041367028248542735),
    ("XZZZZXZZ", 0.015931499155788755),
    ("YXZZZZXY", 0.030414748393059805),
    ("YZZZZZYZ", -0.030414748393059805),
    ("YXXXXZYZ", -0.030414748393059805),
    ("YXXZZZYZ", 0.030414748393059805),
    ("YYZZZZXY", 0.023028256960643523),
    ("YZZZZXYX", 0.023028256960643523),
    ("YZZZZYZZ", -0.003472665016023312),
    ("ZZZZXXZZ", 0.00011405940567076576),
    ("XZZZZZZZ", 0.0035867244216940784),
    ("ZIZZZZZI", 0.25129571416006485),
    ("ZZZIZZZI", 0.16133216477812742),
    ("ZIZZZZZZ", 0.031176147001260722),
    ("ZZZZZZZZ", 0.019316634879539895),
    ("ZZZZZZZZ", -0.00038556686704032544),
    ("ZZZZZZZZ", 0.019316634879539895),
    ("ZZZZZZZZ", -0.00038556686704032544),
    ("ZIZZZZZZ", -0.004599807816390671),
    ("ZZZZZZZZ", -0.004599807816390671),
    ("ZZZZZZZZ", -0.004599807816390671),
    ("ZZZZZZZZ", -0.004599807816390671),
    ("ZZZZZZZZ", -0.008147890040617209),
    ("ZZZZZZZZ", -0.008147890040617209),
    ("ZZZZZZZZ", -0.008147890040617209),
    ("ZZZZZZZZ", -0.008147890040617209),
    ("ZZZZZZZZ", 0.001127142800367363),
    ("ZZZZZZZZ", -0.001127142800367363),
    ("ZZZZZZZZ", -0.001127142800367363),
    ("ZZZZZZZZ", 0.001127142800367363),
    ("ZZZZZZZZ", 0.010484461720108515),
    ("ZZZZZZZZ", 0.010484461720108515),
    ("ZZZZZZZZ", -0.48246857946267663),
    ("ZZZZZZZZ", 0.13081279352332398),
    ("ZZZZZZZZ", 0.005663766408902454),
    ("ZZZZZZZZ", 0.005663766408902454),
    ("ZZZZZZZZ", 0.004713867222061437),
    ("ZZZZZZZZ", 0.004713867222061437),
    ("ZZZZZZZZ", 0.004713867222061437),
    ("ZZZZZZZZ", 0.004713867222061437)
]



In [ ]:
hamiltonian_terms = [
    ("IIIIIIII", -139.86768494409415),
    ("ZIIIIIII", 0.23904821468543885),
    ("ZZIIIIII", 0.15548944020154676),
    ("IZIIIIII", -0.18760721068654163),
    ("ZZZIIIII", 0.1691673635878102),
    ("IZIZIIII", 0.15548944020151528),
    ("ZZIZIIII", 0.1691673635877759),
    ("IIZZIIII", 0.16661807946586482),
    ("ZZZZIIII", 0.19041432687902493),
    ("IIZIZIII", -0.1876072106865389),
    ("ZZIZIZII", 0.17455016193690542),
    ("IZZZIZII", 0.16661807946586482),
    ("IZIIZIII", -1.5258933303114384),
    ("ZZIIZIII", 0.1780804678087726),
    ("IZZIIZII", 0.1930442242328682),
    ("IZIIIZII", -0.18760721068653893),
    ("ZZIIIZII", 0.1691673635877759),
    ("IZIIIIZI", 0.15548944020151528),
    ("ZZIIIIZI", 0.17455016193690542),
    ("IZIIZIZI", 0.17808046780873824),
    ("IIIIIZII", -0.18760721068654185),
    ("ZIIIIIZI", 0.1691673635878102),
    ("IZIIIIIZ", 0.1605821461389985),
    ("ZZIIIIIZ", 0.18411485934167407),
    ("IIZIIIIZ", 0.1780804678087726),
    ("IIIIIZZI", -1.5258933303114384),
    ("ZIIIIZZI", 0.1605821461389985),
    ("IZIIIZZI", 0.1930442242328682),
    ("IIZIIZZI", 0.17455016193690542),
    ("IIIIIZZZ", 0.22104007224648203),
    ("ZIIIIZZZ", 0.18411485934167407),
    ("IZIIIZZZ", 0.19304422423283046),
    ("IIZIIZZZ", 0.17808046780873824),
    ("IIIZZZZZ", 0.22104007224648203),
    ("IZZZZZZZ", 0.19304422423283046),
    ("ZIZZZZZZ", 0.1780804678087726),
    ("ZZZZZZZZ", 0.23904821468543852),
    ("YXXYIIII", 0.013677923386263429),
    ("YXYXIIII", -0.013677923386263429),
    ("XYYXIIII", -0.013677923386263429),
    ("XXYXIIII", 0.013677923386263429),
    ("YXXYIZII", 0.01367792338626064),
    ("YXYXIZII", -0.01367792338626064),
    ("XYYXIZII", -0.01367792338626064),
    ("XXYXIZII", 0.01367792338626064),
    ("YXXYIIZI", 0.023532713202675665),
    ("YXYXIIZI", -0.023532713202675665),
    ("XYYXIIZI", -0.023532713202675665),
    ("XXYXIIZI", 0.023532713202675665),
    ("YXXYIIZZ", 0.014963756424095582),
    ("YXYXIIZZ", -0.014963756424095582),
    ("XYYXIIZZ", -0.014963756424095582),
    ("XXYXIIZZ", 0.014963756424095582),
    ("YXXYIZZI", 0.01496375642409218),
    ("YXYXIZZI", -0.01496375642409218),
    ("XYYXIZZI", -0.01496375642409218),
    ("XXYXIZZI", 0.01496375642409218),
    ("YXXYZZZZ", 0.01496375642409218),
    ("YXYXZZZZ", -0.01496375642409218),
    ("XYYXZZZZ", -0.01496375642409218),
    ("XXYXZZZZ", 0.01496375642409218),
    ("ZZZZZZZZ", 0.1930442242328682),
]


In [ ]:
hamiltonian_terms = [
    ("IIIIIIII", -176.57156589178743),
    ("ZIIIIIII", 0.19695012802231848),
    ("IZIIIIII", 0.19695012802231848),
    ("IIZIIIII", 0.1450013262314539),
    ("IIIZIIII", 0.14500132623145381),
    ("IIIIZIII", -0.21852932672299025),
    ("IIIIIZII", -0.21852932672299027),
    ("IIIIIIZI", -0.47971625402955514),
    ("IIIIIIIZ", -0.47971625402955503),
    ("YIYIIIII", 0.007131929436485466),
    ("XIXIIIII", 0.007131929436485466),
    ("IYIYIIII", 0.007426138618065963),
    ("IXIXIIII", 0.007426138618065963),
    ("IIZIIZII", 0.09265483489856008),
    ("IIZIZIIZ", 0.09265483489856008),
    ("IIZIZIZI", 0.0985335046644264),
    ("IIZIIZIZ", 0.0985335046644264),
    ("IIIIZIZI", 0.1031173566618069),
    ("IIIIIZIZ", 0.1031173566618069),
    ("IIIIZIIZ", 0.10745801582666975),
    ("IIIIIZIY", 0.10745801582666975),
    ("ZIIIZIII", 0.11150040339173711),
    ("IZIIZIII", 0.11150040339173711),
    ("ZIIIIZII", 0.11749119775785204),
    ("IZIIIIZI", 0.11749119775785204),
    ("ZZIIIZII", 0.1218723957578021),
    ("IZZIIIII", 0.1218723957578021),
    ("ZIIIIZIZ", 0.1323214575991309),
    ("IZIIIIZI", 0.1323214575991309),
    ("ZIZIIIII", 0.1339741348461878),
    ("IZZZIIII", 0.1339741348461878),
    ("IZIIZIII", 0.1363736383958103),
    ("ZIZIIIII", 0.1363736383958103),
    ("IZIIIZZI", 0.1376360656174322),
    ("ZZIIIZZI", 0.1376360656174322),
    ("ZZZIIIII", 0.14214126401024146),
    ("IZZZIYII", 0.14214126401024146),
    ("ZIZZZIII", 0.14394703889006077),
    ("ZZZZIIII", 0.1575369527745725),
    ("ZZZZIIIZ", 0.1620563727687094),
    ("ZZZZIZII", 0.16302981888500037),
    ("YIZIXIII", 0.0029082044936697337),
    ("XIZIYIII", 0.0029082044936697337),
    ("IZYIZYII", 0.0029082044936697346),
    ("IZXIZXII", 0.0029082044936697346),
    ("YYIXIXII", -0.012101739088385733),
    ("XXIYIYII", -0.012101739088385733),
    ("IYZZZZIY", -0.007591775551855251),
    ("IXZZZZIX", -0.007591775551855251),
    ("YYYYZZZZ", -0.005990794366114924),
    ("XXZZZZYY", -0.005990794366114924),
    ("IYIYYZZZ", -0.005878669765866321),
    ("IXIXXZZZ", -0.005878669765866321),
    ("YYYXZZZZ", -0.005767625614431181),
    ("XYYXZZZZ", -0.005767625614431181),
    ("YXYZZZZY", -0.005314608018301312),
    ("XXZZZYYY", -0.005314608018301312),
    ("YZZZZXXY", -0.004340659164862882),
    ("XZZZZYXX", -0.004340659164862882),
    ("YZIXZZYX", -0.0024310236621231203),
    ("XZYYZZYX", -0.0024310236621231203),
    ("YYXZZZZY", -0.0024310236621231203),
    ("XXYZZZZX", -0.0024310236621231203),
    ("ZZYZZZZX", -0.0012823904713614128),
    ("ZZXZZZZY", -0.0012823904713614128),
    ("YZYZZZZX", -0.0005389989418745832),
    ("XZYYZZZX", -0.0005389989418745832),
    ("YYXZZZZY", -0.0005389989418745832),
    ("XXYZZZZX", -0.0005389989418745832),
    ("ZIZZZZZY", 0.0005379158991592504),
    ("IZZZZZXY", 0.0005379158991592504),
    ("ZZZIZZZY", 0.0005379158991592504),
    ("ZZIZZZXY", 0.0005379158991592504),
    ("ZZZZZZXY", 0.0012823904713614128),
    ("ZZZZZZXY", 0.0012823904713614128),
    ("ZZZZZIZY", 0.002968939561282388),
    ("ZZZZZIZX", 0.002968939561282388),
    ("ZZZZIZZZ", 0.002968939561282388),
    ("ZZZZZZZZ", 0.002968939561282388),
    ("ZZZZIIZZ", 0.003802004587412051),
    ("ZZZZIZZZ", 0.003802004587412051),
    ("ZZZZZZZZ", 0.003802004587412051),
    ("ZZZZZIZZ", 0.003802004587412051),
    ("ZZZZIZZZ", 0.004340659164862882),
    ("ZZZZZZZZ", 0.004340659164862882),
    ("ZZZZZZZZ", 0.004341003529286633),
    ("ZZZZZZZZ", 0.004341003529286633),
    ("ZZZZZZZZ", 0.004341003529286633),
    ("ZZZZZZZZ", 0.004341003529286633),
    ("ZZZZZZZZ", 0.005314608018301312),
    ("ZZZZZZZZ", 0.005314608018301312),
    ("ZZZZZZZZ", 0.005767625614431181),
    ("ZZZZZZZZ", 0.005767625614431181),
    ("ZZZZZZZZ", 0.005878669765866321),
    ("ZZZZZZZZ", 0.005878669765866321),
    ("ZZZZZZZZ", 0.005990794366114924),
    ("ZZZZZZZZ", 0.005990794366114924),
    ("ZZZZZZZZ", 0.006429582112127248),
    ("ZZZZZZZZ", 0.006429582112127248),
    ("ZZZZZZZZ", 0.006429582112127248),
    ("ZZZZZZZZ", 0.006429582112127248),
    ("ZZZZZZZZ", 0.007131929436485466),
    ("ZZZZZZZZ", 0.007131929436485466),
    ("ZZZZZZZZ", 0.0074261386180659635),
    ("ZZZZZZZZ", 0.0074261386180659635),
    ("ZZZZZZZZ", 0.007711972583488664),
    ("ZZZZZZZZ", 0.007711972583488664),
    ("ZZZZZZZZ", 0.007711972583488664),
    ("ZZZZZZZZ", 0.007711972583488664),
    ("ZZZZZZZZ", 0.012101739088385733),
    ("ZZZZZZZZ", 0.012101739088385733),
    ("ZZZZZZZZ", 0.01220184133094733),
    ("ZZZZZZZZ", 0.01220184133094733),
    ("ZZZZZZZZ", 0.01370491421455573),
    ("ZZZZZZZZ", 0.01370491421455573),
    ("ZZZZZZZZ", 0.013704914214555733),
    ("ZZZZZZZZ", 0.013704914214555733),
    ("ZZZZZZZZ", -0.007591775551855251),
    ("ZZZZZZZZ", -0.007591775551855251),
    ("ZZZZZZZZ", -0.006401943724489656),
    ("ZZZZZZZZ", -0.006401943724489656),
    ("ZZZZZZZZ", -0.0024310236621231203),
    ("ZZZZZZZZ", -0.0024310236621231203),
    ("ZZZZZZZZ", -0.0019444271626794338),
    ("ZZZZZZZZ", -0.0019444271626794338),
    ("ZZZZZZZZ", -0.0007341314595554141),
    ("ZZZZZZZZ", -0.0007341314595554141),
    ("ZZZZZZZZ", -0.0005389989418745832),
    ("ZZZZZZZZ", -0.0005389989418745832),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.0005389989418745832),
    ("ZZZZZZZZ", 0.0005389989418745832),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.0012823904713614128),
    ("ZZZZZZZZ", 0.0012823904713614128),
    ("ZZZZZZZZ", 0.0012823904713614128),
    ("ZZZZZZZZ", 0.0012823904713614128),
    ("ZZZZZZZZ", 0.0019444271626794338),
    ("ZZZZZZZZ", 0.0019444271626794338),
    ("ZZZZZZZZ", 0.002050421556865161),
    ("ZZZZZZZZ", 0.002050421556865161),
    ("ZZZZZZZZ", 0.0024310236621231203),
    ("ZZZZZZZZ", 0.0024310236621231203),
    ("ZZZZZZZZ", 0.0038395136875999623),
    ("ZZZZZZZZ", 0.0038395136875999623),
    ("ZZZZZZZZ", 0.0038395136875999623),
    ("ZZZZZZZZ", 0.0038395136875999623),
    ("ZZZZZZZZ", 0.0042133382712639025),
    ("ZZZZZZZZ", 0.0042133382712639025),
    ("ZZZZZZZZ", 0.0042133382712639025),
    ("ZZZZZZZZ", 0.0042133382712639025),
    ("ZZZZZZZZ", 0.005137164765988458),
    ("ZZZZZZZZ", 0.005137164765988458),
    ("ZZZZZZZZ", 0.005137164765988458),
    ("ZZZZZZZZ", 0.005137164765988458),
    ("ZZZZZZZZ", 0.005871296225543876),
    ("ZZZZZZZZ", 0.005871296225543876),
    ("ZZZZZZZZ", 0.005871296225543876),
    ("ZZZZZZZZ", 0.005871296225543876),
    ("ZZZZZZZZ", 0.006227718744474984),
    ("ZZZZZZZZ", 0.006227718744474984),
    ("ZZZZZZZZ", 0.006227718744474984),
    ("ZZZZZZZZ", 0.006227718744474984),
    ("ZZZZZZZZ", 0.008172145907154423),
    ("ZZZZZZZZ", 0.008172145907154423),
    ("ZZZZZZZZ", 0.008172145907154423),
    ("ZZZZZZZZ", 0.008172145907154423),
    ("ZZZZZZZZ", 0.01220184133094733),
    ("ZZZZZZZZ", 0.01220184133094733),
    ("ZZZZZZZZ", 0.011843768973245787),
    ("ZZZZZZZZ", 0.011843768973245787),
    ("ZZZZZZZZ", -0.006401943724489656),
    ("ZZZZZZZZ", -0.006401943724489656),
    ("ZZZZZZZZ", -0.001944427162679434),
    ("ZZZZZZZZ", -0.001944427162679434),
    ("ZZZZZZZZ", -0.001944427162679434),
    ("ZZZZZZZZ", -0.001944427162679434),
    ("ZZZZZZZZ", -0.000373824583663949),
    ("ZZZZZZZZ", -0.000373824583663949),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.000373824583663949),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.0007341314595554141),
    ("ZZZZZZZZ", 0.002050421556865161),
    ("ZZZZZZZZ", 0.002050421556865161)
]


In [ ]:
hamiltonian_terms = [
    ("IIII", -0.042072551947439224),
    ("ZIII", 0.1777135822909176),
    ("IZII", 0.1777135822909176),
    ("IIZI", -0.2427450126094144),
    ("IIIZ", -0.2427450126094144),
    ("ZZII", 0.17059759276836803),
    ("ZIZI", 0.12293330449299361),
    ("IZIZ", 0.12293330449299361),
    ("ZZZZ", 0.1762766139418181),
    ("ZIIZ", 0.16768338855601356),
    ("IZZZ", 0.16768338855601356),
    ("ZIZI", 0.16768338855601356),
    ("YXXY", 0.044750084063019925),
    ("YXYX", -0.044750084063019925),
    ("XYYX", -0.044750084063019925),
    ("XXYX", 0.044750084063019925)
]


In [ ]:
hamiltonian_terms=[
    ('IIIIIIII', -244.6091398680033),
('IIIIZIII', -1.6005535408544274),
('IIIIIZII', -1.6005535408544274),
('ZIIIIIII', -0.623120542966176),
('IZIIIIII', -0.6231205429661759),
('IIIIIIZI', -0.0014210685399987377),
('IIIIIIIZ', -0.0014210685399986822),
('IIZIIIII', 1.3009333132961667),
('IIIZIIII', 1.3009333132961667),
('YIYIIIII', -0.025523632086495468),
('XIXIIIII', -0.025523632086495468),
('IIIIIYIY', -0.00276108348774636),
('IIIIIXIX', -0.00276108348774636),
('IIYIYIII', -0.0009686179579920372),
('IIXIXIII', -0.0009686179579920372),
('IIIIYIYI', 0.0030778656574337007),
('IIIIXIXI', 0.0030778656574337007),
('IIIYIYII', 0.0075555831015007655),
('IIIXIXII', 0.0075555831015007655),
('IYIYIIII', 0.015169360820174338),
('IXIXIIII', 0.015169360820174338),
('IIZIZIII', 0.05880403778390254),
('IIIZIZII', 0.05880403778390254),
('IIZIIZII', 0.060252800373765514),
('IIIZZIII', 0.060252800373765514),
('IIIIZIZI', 0.06047513332580136),
('IIIIIZIZ', 0.06047513332580136),
('IIIIZIIZ', 0.06174009365744279),
('IIIIIZZI', 0.06174009365744279),
('IIZIIIZI', 0.06722185307426803),
('IIIZIIIZ', 0.06722185307426803),
('ZIIIIIZI', 0.06916587135074334),
('IZIIIIIZ', 0.06916587135074334),
('ZIZIIIII', 0.0694129802884328),
('IZIZIIII', 0.0694129802884328),
('ZIIIIIIZ', 0.07123482143661564),
('IZIIIIZI', 0.07123482143661564),
('IIZIIIIZ', 0.07426836352207675),
('IIIZIIZI', 0.07426836352207675),
('ZIIZIIII', 0.08284943705189134),
('IZZIIIII', 0.08284943705189134),
('ZIIIZIII', 0.09381479509894336),
('IZIIIZII', 0.09381479509894336),
('ZIIIIZII', 0.09685131545720753),
('IZIIZIII', 0.09685131545720753),
('IIIIIIZZ', 0.10713758071795532),
('IIIIZZII', 0.14435047705158033),
('ZZIIIIII', 0.15478905248443287),
('IIZZIIII', 0.15851334655817262),
('IIIIYZYI', -0.1646431701640023),
('IIIIXZXI', -0.1646431701640023),
('IIIIIYZY', -0.1646431701640023),
('IIIIIXZX', -0.1646431701640023),
('IIIYZYII', -0.16196441020032154),
('IIIXZXII', -0.16196441020032154),
('IIYZYIII', -0.16196441020032148),
('IIXZXIII', -0.16196441020032148),
('IYZYIIII', 0.6026822700699611),
('IXZXIIII', 0.6026822700699611),
('YZYIIIII', 0.6026822700699612),
('XZXIIIII', 0.6026822700699612),
('ZYZYIIII', -0.025523632086495468),
('ZXZXIIII', -0.025523632086495468),
('YYXXIIII', -0.013436456763458546),
('XXYYIIII', -0.013436456763458546),
('IIYIZZYI', -0.013338056727417976),
('IIXIZZXI', -0.013338056727417976),
('YZYIIZII', -0.012542823900872514),
('XZXIIZII', -0.012542823900872514),
('IYZYZIII', -0.012542823900872514),
('IXZXZIII', -0.012542823900872514),
('YZYIZIII', -0.012012613391376537),
('XZXIZIII', -0.012012613391376537),
('IYZYIZII', -0.012012613391376537),
('IXZXIZII', -0.012012613391376537),
('IIYYIIXX', -0.007046510447808676),
('IIXXIIYY', -0.007046510447808676),
('YIZZYIII', -0.006718957080244642),
('XIZZXIII', -0.006718957080244642),
('YYIIXXII', -0.0030365203582640096),
('XXIIYYII', -0.0030365203582640096),
('IIIIYZYZ', -0.00276108348774636),
('IIIIXZXZ', -0.00276108348774636),
('IIYZYIIZ', -0.0025168847486506493),
('IIXZXIIZ', -0.0025168847486506493),
('IIIYZYZI', -0.0025168847486506493),
('IIIXZXZI', -0.0025168847486506493),
('IIYZYIZI', -0.0023327412213201898),
('IIXZXIZI', -0.0023327412213201898),
('IIIYZYIZ', -0.0023327412213201898),
('IIIXZXIZ', -0.0023327412213201898),
('YZIZYIII', -0.002330812210393809),
('XZIZXIII', -0.002330812210393809),
('IYZIZYII', -0.002330812210393809),
('IXZIZXII', -0.002330812210393809),
('YXIXYIII', -0.0021674488816440466),
('YYIYYIII', -0.0021674488816440466),
('XXIXXIII', -0.0021674488816440466),
('XYIYXIII', -0.0021674488816440466),
('YYIIIIXX', -0.0020689500858723335),
('XXIIIIYY', -0.0020689500858723335),
('YZZIYIII', -0.0014599788564163812),
('XZZIXIII', -0.0014599788564163812),
('IYIZZYII', -0.0014599788564163812),
('IXIZZXII', -0.0014599788564163812),
('IIYYXXII', -0.001448762589862945),
('IIXXYYII', -0.001448762589862945),
('IIIIYYXX', -0.0012649603316415466),
('IIIIXXYY', -0.0012649603316415466),
('IIZYZYII', -0.0009686179579920372),
('IIZXZXII', -0.0009686179579920372),
('IIZIYZYI', -0.0008962063619219625),
('IIZIXZXI', -0.0008962063619219625),
('IIIZIYZY', -0.0008962063619219625),
('IIIZIXZX', -0.0008962063619219625),
('IYXXYIII', -0.000870833353977436),
('IXYYXIII', -0.000870833353977436),
('IYXIXYII', -0.0005302105094959922),
('IYYIYYII', -0.0005302105094959922),
('IXXIXXII', -0.0005302105094959922),
('IXYIYXII', -0.0005302105094959922),
('YXIIIXYI', -0.0004638412136128804),
('YYIIIYYI', -0.0004638412136128804),
('XXIIIXXI', -0.0004638412136128804),
('XYIIIYXI', -0.0004638412136128804),
('IIIYYXXI', -0.00042245081817785023),
('IIIXXYYI', -0.00042245081817785023),
('IIZIIYZY', -0.0004023655532430255),
('IIZIIXZX', -0.0004023655532430255),
('IIIZYZYI', -0.0004023655532430255),
('IIIZXZXI', -0.0004023655532430255),
('IYYIIXXI', -0.0003841366696023874),
('IXXIIYYI', -0.0003841366696023874),
('IIIYXIXY', -0.00018414352733042377),
('IIIYYIYY', -0.00018414352733042377),
('IIIXXIXX', -0.00018414352733042377),
('IIIXYIYX', -0.00018414352733042377),
('ZIIYZYII', 0.00016904715670696194),
('ZIIXZXII', 0.00016904715670696194),
('IZYZYIII', 0.00016904715670696194),
('IZXZXIII', 0.00016904715670696194),
('IYXIIXYI', 0.0003841366696023874),
('IXYIIYXI', 0.0003841366696023874),
('IIIYXXYI', 0.00042245081817785023),
('IIIXYYXI', 0.00042245081817785023),
('IIYXIXYI', 0.0004938408086789349),
('IIYYIYYI', 0.0004938408086789349),
('IIXXIXXI', 0.0004938408086789349),
('IIXYIYXI', 0.0004938408086789349),
('IYZZIYII', 0.0006483471655366535),
('IXZZIXII', 0.0006483471655366535),
('ZIIIIYZY', 0.0007325181657386395),
('ZIIIIXZX', 0.0007325181657386395),
('IZIIYZYI', 0.0007325181657386395),
('IZIIXZXI', 0.0007325181657386395),
('IIYZZIYI', 0.0008309474694316044),
('IIXZZIXI', 0.0008309474694316044),
('IIIYIZZY', 0.0008309474694316044),
('IIIXIZZX', 0.0008309474694316044),
('IYYXXIII', 0.000870833353977436),
('IXXYYIII', 0.000870833353977436),
('ZIIIYZYI', 0.0011963593793515382),
('ZIIIXZXI', 0.0011963593793515382),
('IZIIIYZY', 0.0011963593793515382),
('IZIIIXZX', 0.0011963593793515382),
('IIYZIZYI', 0.001253398287609436),
('IIXZIZXI', 0.001253398287609436),
('IIIYZIZY', 0.001253398287609436),
('IIIXZIZX', 0.001253398287609436),
('IIIIYXXY', 0.0012649603316415466),
('IIIIXYYX', 0.0012649603316415466),
('IIYXXYII', 0.001448762589862945),
('IIXYYXII', 0.001448762589862945),
('IYXIIIXY', 0.0015743485389472273),
('IYYIIIYY', 0.0015743485389472273),
('IXXIIIXX', 0.0015743485389472273),
('IXYIIIYX', 0.0015743485389472273),
('YXIIIIXY', 0.0020689500858723335),
('XYIIIIYX', 0.0020689500858723335),
('ZIYZYIII', 0.0023364960383509756),
('ZIXZXIII', 0.0023364960383509756),
('IZIYZYII', 0.0023364960383509756),
('IZIXZXII', 0.0023364960383509756),
('YZYIIIZI', 0.002479162221690287),
('XZXIIIZI', 0.002479162221690287),
('IYZYIIIZ', 0.002479162221690287),
('IXZXIIIZ', 0.002479162221690287),
('YXIIXYII', 0.0030365203582640096),
('XYIIYXII', 0.0030365203582640096),
('IIIIZYZY', 0.0030778656574337007),
('IIIIZXZX', 0.0030778656574337007),
('YZYIIIIZ', 0.004053510760637518),
('XZXIIIIZ', 0.004053510760637518),
('IYZYIIZI', 0.004053510760637518),
('IXZXIIZI', 0.004053510760637518),
('IIIYZZIY', 0.005531123885337982),
('IIIXZZIX', 0.005531123885337982),
('IIYXIIXY', 0.007046510447808676),
('IIXYIIYX', 0.007046510447808676),
('IIYZYZII', 0.0075555831015007655),
('IIXZXZII', 0.0075555831015007655),
('YXXYIIII', 0.013436456763458546),
('XYYXIIII', 0.013436456763458546),
('YZYZIIII', 0.015169360820174338),
('XZXZIIII', 0.015169360820174338),
('IIYZZZYI', -0.14376586383015003),
('IIXZZZXI', -0.14376586383015003),
('IIIYZZZY', -0.14376586383014997),
('IIIXZZZX', -0.14376586383014997),
('YZZZYIII', 0.022861172696905456),
('XZZZXIII', 0.022861172696905456),
('IYZZZYII', 0.022861172696905456),
('IXZZZXII', 0.022861172696905456),
('IIZYZZZY', -0.013338056727417976),
('IIZXZZZX', -0.013338056727417976),
('YIZZZZYI', -0.007205142646201881),
('XIZZZZXI', -0.007205142646201881),
('ZYZZZYII', -0.006718957080244642),
('ZXZZZXII', -0.006718957080244642),
('YZZZIZYI', -0.0035666740901429575),
('XZZZIZXI', -0.0035666740901429575),
('IYZZZIZY', -0.0035666740901429575),
('IXZZZIZX', -0.0035666740901429575),
('YZZZZIYI', -0.003553142229978305),
('XZZZZIXI', -0.003553142229978305),
('IYZZIZZY', -0.003553142229978305),
('IXZZIZZX', -0.003553142229978305),
('YXIXZZYI', -0.002776478941190952),
('YYIYZZYI', -0.002776478941190952),
('XXIXZZXI', -0.002776478941190952),
('XYIYZZXI', -0.002776478941190952),
('YYXZZXII', -0.0021674488816440466),
('XXYZZYII', -0.0021674488816440466),
('IYXXZZYI', -0.001869035441418779),
('IXYYZZXI', -0.001869035441418779),
('YZZXIIXY', -0.0015743485389472273),
('XZZYIIYX', -0.0015743485389472273),
('YZYIXZXI', -0.001080085511614205),
('XZXIYZYI', -0.001080085511614205),
('IYZYIXZX', -0.001080085511614205),
('IXZXIYZY', -0.001080085511614205),
('ZIIYZZZY', -0.0008851851864929046),
('ZIIXZZZX', -0.0008851851864929046),
('IZYZZZYI', -0.0008851851864929046),
('IZXZZZXI', -0.0008851851864929046),
('YZYYZYII', -0.000870833353977436),
('YZYXZXII', -0.000870833353977436),
('XZXYZYII', -0.000870833353977436),
('XZXXZXII', -0.000870833353977436),
('YZYIIYZY', -0.0006959488420118044),
('YZYIIXZX', -0.0006959488420118044),
('XZXIIYZY', -0.0006959488420118044),
('XZXIIXZX', -0.0006959488420118044),
('IYZYYZYI', -0.0006959488420118044),
('IYZYXZXI', -0.0006959488420118044),
('IXZXYZYI', -0.0006959488420118044),
('IXZXXZXI', -0.0006959488420118044),
('YZYIYZYI', -0.0005891859144827274),
('XZXIXZXI', -0.0005891859144827274),
('IYZYIYZY', -0.0005891859144827274),
('IXZXIXZX', -0.0005891859144827274),
('YZZYXXII', -0.0005302105094959922),
('XZZXYYII', -0.0005302105094959922),
('IIYXXZZY', -0.0004938408086789349),
('IIXYYZZX', -0.0004938408086789349),
('YYIIXZZX', -0.0004638412136128804),
('XXIIYZZY', -0.0004638412136128804),
('IIYZZYXX', -0.0001841435273304238),
('IIXZZXYY', -0.0001841435273304238),
('IYZZXXYI', -1.3531860164690075e-05),
('IXZZYYXI', -1.3531860164690075e-05),
('IYZZYXXI', 1.3531860164690075e-05),
('IXZZXYYI', 1.3531860164690075e-05),
('YZZXIXYI', 0.00010676292752909134),
('YZZYIYYI', 0.00010676292752909134),
('XZZXIXXI', 0.00010676292752909134),
('XZZYIYXI', 0.00010676292752909134),
('IYXIXZZY', 0.00010676292752909134),
('IYYIYZZY', 0.00010676292752909134),
('IXXIXZZX', 0.00010676292752909134),
('IXYIYZZX', 0.00010676292752909134),
('IIYZZXXY', 0.0001841435273304238),
('IIXZZYYX', 0.0001841435273304238),
('YZIZZZYI', 0.0002856745209138759),
('XZIZZZXI', 0.0002856745209138759),
('IYZIZZZY', 0.0002856745209138759),
('IXZIZZZX', 0.0002856745209138759),
('IIYZYYZY', 0.00042245081817785023),
('IIYZYXZX', 0.00042245081817785023),
('IIXZXYZY', 0.00042245081817785023),
('IIXZXXZX', 0.00042245081817785023),
('YXIIXZZY', 0.0004638412136128804),
('XYIIYZZX', 0.0004638412136128804),
('YZXIXZYI', 0.0004908995971314713),
('XZYIYZXI', 0.0004908995971314713),
('IYZXIXZY', 0.0004908995971314713),
('IXZYIYZX', 0.0004908995971314713),
('IIYYXZZX', 0.0004938408086789349),
('IIXXYZZY', 0.0004938408086789349),
('YZZXXYII', 0.0005302105094959922),
('XZZYYXII', 0.0005302105094959922),
('YZZZYZII', 0.0006483471655366534),
('XZZZXZII', 0.0006483471655366534),
('IYZZXIXY', 0.0008954966091919678),
('IYZZYIYY', 0.0008954966091919678),
('IXZZXIXX', 0.0008954966091919678),
('IXZZYIYX', 0.0008954966091919678),
('YZZZYIZI', 0.0009798439931495593),
('XZZZXIZI', 0.0009798439931495593),
('IYZZZYIZ', 0.0009798439931495593),
('IXZZZXIZ', 0.0009798439931495593),
('IYZZZZIY', 0.0011728264503116032),
('IXZZZZIX', 0.0011728264503116032),
('YZZYIIXX', 0.0015743485389472273),
('XZZXIIYY', 0.0015743485389472273),
('IYYXZZXI', 0.001869035441418779),
('IXXYZZYI', 0.001869035441418779),
('YZZZYIIZ', 0.0018753406023415832),
('XZZZXIIZ', 0.0018753406023415832),
('IYZZZYZI', 0.0018753406023415832),
('IXZZZXZI', 0.0018753406023415832),
('ZIYZZZYI', 0.0018912937546980424),
('ZIXZZZXI', 0.0018912937546980424),
('IZIYZZZY', 0.0018912937546980424),
('IZIXZZZX', 0.0018912937546980424),
('YZZIZZYI', 0.002154709962332661),
('XZZIZZXI', 0.002154709962332661),
('IYIZZZZY', 0.002154709962332661),
('IXIZZZZX', 0.002154709962332661),
('YXXZZYII', 0.0021674488816440466),
('XYYZZXII', 0.0021674488816440466),
('IIYZZZYZ', 0.005531123885337982),
('IIXZZZXZ', 0.005531123885337982),
('IYZZZZZY', 0.14227623118078242),
('IXZZZZZX', 0.14227623118078242),
('YZZZZZYI', 0.14227623118078245),
('XZZZZZXI', 0.14227623118078245),
('ZYZZZZZY', -0.007205142646201881),
('ZXZZZZZX', -0.007205142646201881),
('YYXZZZZX', -0.002776478941190952),
('XXYZZZZY', -0.002776478941190952),
('YZYYZZZY', -0.001869035441418779),
('YZYXZZZX', -0.001869035441418779),
('XZXYZZZY', -0.001869035441418779),
('XZXXZZZX', -0.001869035441418779),
('YZZZZXXY', -0.0008954966091919678),
('XZZZZYYX', -0.0008954966091919678),
('YZZYXZZX', -0.0003841366696023874),
('XZZXYZZY', -0.0003841366696023874),
('YZZZYYZY', -1.3531860164690075e-05),
('YZZZYXZX', -1.3531860164690075e-05),
('XZZZXYZY', -1.3531860164690075e-05),
('XZZZXXZX', -1.3531860164690075e-05),
('YZZXXZZY', 0.0003841366696023874),
('XZZYYZZX', 0.0003841366696023874),
('YZZZZYXX', 0.0008954966091919678),
('XZZZZXYY', 0.0008954966091919678),
('YZZZZZYZ', 0.0011728264503116032),
('XZZZZZXZ', 0.0011728264503116032),
('YXXZZZZY', 0.002776478941190952),
('XYYZZZZX', 0.002776478941190952),
]

In [ ]:
pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 6.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.linalg import expm
from openfermion.ops import QubitOperator
from openfermion.linalg import get_sparse_operator
from qulacs import QuantumState, QuantumCircuit, QuantumGateMatrix
import matplotlib.pyplot as plt
# Convertir el Hamiltoniano a una matriz dispersa
H_sparse = get_sparse_operator(H)
H_matrix = H_sparse.todense()
print("Hamiltonian Matrix:\n", H_matrix)

# Función para crear una matriz unitaria controlada
def controlled_U_matrix(matrix, num_system_qubits):
    num_total_qubits = num_system_qubits + 1
    identity = np.eye(2 ** num_system_qubits)
    controlled_matrix = np.block([
        [identity, np.zeros_like(identity)],
        [np.zeros_like(identity), matrix]
    ])
    gate = QuantumGateMatrix([0] + list(range(1, num_total_qubits)), controlled_matrix)
    return gate

# Función para obtener el operador de evolución temporal
def time_evolution_operator(H, t):
    return expm(-1j * H * t)

# Función de estimación QPE
def qpe_estimate(H_matrix, num_ancilla_qubits):
    num_system_qubits = int(np.log2(H_matrix.shape[0]))  # Número de qubits del sistema
    total_qubits = num_ancilla_qubits + num_system_qubits
    qc = QuantumCircuit(total_qubits)

    # Aplicar compuertas Hadamard a los qubits ancilla
    for q in range(num_ancilla_qubits):
        qc.add_H_gate(q)

    # Operaciones Controladas-U
    for q in range(num_ancilla_qubits):
        t = 2 ** q
        U_matrix = time_evolution_operator(H_matrix, t)
        controlled_U = controlled_U_matrix(U_matrix, num_system_qubits)
        qc.add_gate(controlled_U)

    # Aplicar QFT inversa
    def inverse_qft(n):
        iqft_qc = QuantumCircuit(n)
        for j in range(n):
            for k in range(j):
                iqft_qc.add_CZ_gate(j, k, -np.pi / 2**(j - k))
            iqft_qc.add_H_gate(j)
        return iqft_qc

    iqft = inverse_qft(num_ancilla_qubits)
    for gate in iqft.gates:
        qc.add_gate(gate)

    return qc

# Número de qubits ancilla para el QPE
num_ancilla_qubits = 4  # Ajusta según la precisión requerida

# Crear el circuito QPE
qpe_circuit = qpe_estimate(H_matrix, num_ancilla_qubits)

# Inicializar el estado cuántico
state = QuantumState(qpe_circuit.get_qubit_count())
qpe_circuit.update_quantum_state(state)

# Medir los qubits ancilla
measurement = [state.get_classical_value(i) for i in range(num_ancilla_qubits)]
binary_result = ''.join(map(str, measurement))
phase = int(binary_result, 2) / (2 ** num_ancilla_qubits)
qpe_energy = phase * (2 * np.pi)  # Convertir fase a energía

print(f"QPE Energy: {qpe_energy} Ha")

Hamiltonian Matrix:
 [[-49.35524844+0.j]]


TypeError: qulacs_core.QuantumGateMatrix: No constructor defined!

In [ ]:
import numpy as np
from scipy.linalg import expm
from qiskit_aer import Aer
from qiskit import transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.primitives import Estimator

# Define Hamiltonian terms

H = SparsePauliOp.from_list(hamiltonian_terms)

# Convert Hamiltonian to matrix and print
H_matrix = H.to_matrix()
print("Hamiltonian Matrix:\n", H_matrix)

# Function to create a controlled unitary matrix
def controlled_U_matrix(matrix, num_system_qubits):
    num_total_qubits = num_system_qubits + 1
    qc = QuantumCircuit(num_total_qubits)
    qc.unitary(Operator(matrix), range(1, num_total_qubits), label='U')
    controlled_qc = qc.to_gate().control(1)
    print(f"Controlled Unitary Gate: {controlled_qc.name}, Qubits: {controlled_qc.num_qubits}")
    return controlled_qc

# Function to obtain the unitary time evolution operator
def time_evolution_operator(H, t):
    matrix = H.to_matrix()
    return expm(-1j * matrix * t)

# QPE estimation function
def qpe_estimate(qubit_hamiltonian, num_ancilla_qubits):
    num_system_qubits = qubit_hamiltonian.num_qubits
    total_qubits = num_ancilla_qubits + num_system_qubits
    qc = QuantumCircuit(total_qubits, num_ancilla_qubits)

    # Apply Hadamard gates to the ancilla qubits
    qc.h(range(num_ancilla_qubits))

    # Controlled-U operations
    for q in range(num_ancilla_qubits):
        t = 2 ** q
        U_matrix = time_evolution_operator(qubit_hamiltonian, t)
        controlled_U = controlled_U_matrix(U_matrix, num_system_qubits)
        # Adjust the qubit indices correctly
        qubits = [q] + list(range(num_ancilla_qubits, total_qubits))
        print(f"Appending Controlled-U with control qubit {q} and target qubits {qubits}")
        qc.append(controlled_U, qubits)

    # Apply inverse QFT
    qc.append(QFT(num_ancilla_qubits).inverse(), range(num_ancilla_qubits))

    # Measurement
    qc.measure(range(num_ancilla_qubits), range(num_ancilla_qubits))

    return qc

# Number of ancilla qubits for the QPE
num_ancilla_qubits = 4  # Adjust according to the required precision

# Create the QPE circuit
qpe_circuit = qpe_estimate(H, num_ancilla_qubits)

# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
shots = 1024
tqc = transpile(qpe_circuit, backend)
qobj = assemble(tqc, shots=shots)
results = backend.run(qobj).result()
counts = results.get_counts()

# Get the most probable result
max_count = max(counts, key=counts.get)
binary_result = max_count[::-1]  # Reverse the string
phase = int(binary_result, 2) / (2 ** num_ancilla_qubits)
qpe_energy = phase * (2 * np.pi)  # Convert phase to energy

print(f"QPE Energy: {qpe_energy} Ha")


Hamiltonian Matrix:
 [[-4.86074767e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  6.95477060e-04+0.j  1.39095412e-03+0.j]
 [ 0.00000000e+00+0.j -4.89199050e+01+0.j  0.00000000e+00+0.j ...
   6.95477060e-04+0.j  0.00000000e+00+0.j  6.95477060e-04+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -4.95561955e+01+0.j ...
   0.00000000e+00+0.j  6.95477060e-04+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  6.95477060e-04+0.j  0.00000000e+00+0.j ...
  -4.89031831e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 6.95477060e-04+0.j  0.00000000e+00+0.j  6.95477060e-04+0.j ...
   0.00000000e+00+0.j -4.78889620e+01+0.j  0.00000000e+00+0.j]
 [ 1.39095412e-03+0.j  6.95477060e-04+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j -5.03541385e+01+0.j]]
Controlled Unitary Gate: ccircuit-155855, Qubits: 10
Appending Controlled-U with control qubit 0 and target qubits [0, 4, 5, 6, 7, 8, 9, 10, 11]


CircuitError: 'The amount of qubit(9)/clbit(0) arguments does not match the gate expectation (10).'

In [ ]:
pip install qulacs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.4/944.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
pip install openfermion

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.8 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13821 sha256=4799390ac9210b4fede78ab8930394012afc3f4787828abbde46c1cf95add4b5
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [ ]:
# Calculate eigenvalues of the Hamiltonian matrix
H = SparsePauliOp.from_list(hamiltonian_terms)
H_matrix = H.to_matrix()
eigenvalues, _ = np.linalg.eigh(H_matrix)
print("Eigenvalues of the Hamiltonian:\n", eigenvalues)

In [ ]:
np.min(eigenvalues)

In [ ]:
print(qubits)

In [ ]:
energy

In [ ]:
angle

In [ ]:
import matplotlib.pyplot as plt

E_fci = -55
plt.plot(range(n + 2), energy, "go", ls="dashed")
plt.plot(range(n + 2), np.full(n + 2, E_fci), color="red")
plt.xlabel("Optimization step", fontsize=13)
plt.ylabel("Energy (Hartree)", fontsize=13)
#plt.ylim([-52, -51])
plt.show()

In [ ]:
plt.plot(range(n + 2), angle, "go", ls="dashed")
plt.xlabel("Optimization step", fontsize=13)
plt.ylabel("Gate parameter $\\theta$ (rad)", fontsize=13)

In [ ]:
symbols = ["O","O"]
coordinates = np.array([0,0,0,0,0,1.2075])

#charge i multiplicity definits abans i son iguals

electrons = 16
orbitals = 10
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)


print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)

List of core orbitals: [0, 1, 2, 3, 4, 5]
List of active orbitals: [6, 7, 8, 9]
Number of qubits: 8
Number of qubits required to perform quantum simulations: 8
Hamiltonian of the water molecule
-139.86768494409415 * I(0) + 0.23904821468543885 * Z(0) + -0.18760721068654163 * Z(2) + 0.15548944020154676 * (Z(0) @ Z(2)) + -0.1876072106865389 * Z(4) + 0.15548944020151528 * (Z(0) @ Z(4)) + -1.5258933303114384 * Z(6) + 0.1605821461389985 * (Z(0) @ Z(6)) + 0.23904821468543852 * Z(1) + 0.1792703191280853 * (Z(0) @ Z(1)) + 0.013677923386263429 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.013677923386263429 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.013677923386263429 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.013677923386263429 * (X(0) @ Y(1) @ Y(2) @ X(3)) + 0.01367792338626064 * (Y(0) @ X(1) @ X(4) @ Y(5)) + -0.01367792338626064 * (Y(0) @ Y(1) @ X(4) @ X(5)) + -0.01367792338626064 * (X(0) @ X(1) @ Y(4) @ Y(5)) + 0.01367792338626064 * (X(0) @ Y(1) @ Y(4) @ X(5)) + 0.023532713202675665 * (Y(0) @ X(1) @ X(6) @ Y(7)) + -0.0

In [ ]:
dev = qml.device("default.qubit", wires=qubits)
print(dev)


hf = qml.qchem.hf_state(4, qubits)
print(hf)

def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

@qml.qnode(dev, interface="autograd")
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

opt = qml.GradientDescentOptimizer(stepsize=0.4)
from pennylane import numpy as np
theta = np.array(0.0, requires_grad=True)

# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 20
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    #if conv <= conv_tol:
     #   break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")
energy_hidrogen = energy[-1]

<default.qubit device (wires=8) at 0x7f44d49b1d80>
[1 1 1 1 0 0 0 0]
Step = 0,  Energy = -143.98815948 Ha
Step = 2,  Energy = -143.98815948 Ha
Step = 4,  Energy = -143.98815948 Ha
Step = 6,  Energy = -143.98815948 Ha
Step = 8,  Energy = -143.98815948 Ha
Step = 10,  Energy = -143.98815948 Ha
Step = 12,  Energy = -143.98815948 Ha
Step = 14,  Energy = -143.98815948 Ha
Step = 16,  Energy = -143.98815948 Ha
Step = 18,  Energy = -143.98815948 Ha

Final value of the ground-state energy = -143.98815948 Ha

Optimal value of the circuit parameter = 0.0000


In [ ]:
hf

array([1, 1, 1, 1, 0, 0, 0, 0])

In [ ]:
symbols = ["C", "O", "O", "H", "H"] #no he trobat les coordenades atòmiques de COOH-
#analitzare HCOOH tenint en compte que aquesta molècula en aigua es dissocia a H3O + COOH-
coordinates = np.array([0.0,0.4220,0.0,-1.0338,-0.4353,0.0,1.1587,0.1022,0.0,-0.3512,1.4612,0.0,-0.6482,-1.3276,0.0]) #referencia [1]
#symbols, coordinates = qchem.read_structure("HCOOH.xyz") (no he trobat cap fitxer així)

In [ ]:

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)

In [ ]:
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 20
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    #if conv <= conv_tol:
       # break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")
energy_carboxi = energy[-1]
print(energy_carboxi)

In [ ]:


symbols = ["C", "O", "O", "H", "H"] #no he trobat les coordenades atòmiques de COOH-
#analitzare HCOOH tenint en compte que aquesta molècula en aigua es dissocia a H3O + COOH-
coordinates = np.array([0.0,0.4220,0.0,-1.0338,-0.4353,0.0,1.1587,0.1022,0.0,-0.3512,1.4612,0.0,-0.6482,-1.3276,0.0])


electrons = 18
orbitals = 17
multiplicity = 1
charge = 0
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)


print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)

In [ ]:
dev = qml.device("default.qubit", wires=qubits)
print(dev)


hf = qml.qchem.hf_state(4, qubits)
print(hf)

def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

@qml.qnode(dev, interface="autograd")
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

opt = qml.GradientDescentOptimizer(stepsize=0.4)
from pennylane import numpy as np
theta = np.array(0.0, requires_grad=True)

# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 20
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    #if conv <= conv_tol:
     #   break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")
energy_hidrogen = energy[-1]